In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.saving import register_keras_serializable
import os
from tqdm import tqdm

# --- Timing Decorator ---
def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"{func.__name__} executed in {elapsed:.2f} seconds")
        return result
    return wrapper

# --- Custom Loss Functions and Metrics ---

def jaccard_score(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return (intersection + K.epsilon()) / (union + K.epsilon())

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_bce_loss(y_true, y_pred, bce_weight=0.5, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    bce = K.binary_crossentropy(y_true, y_pred)
    return bce_weight * bce + (1 - bce_weight) * dice

def weighted_binary_crossentropy(y_true, y_pred):
    weight = 10
    bce = K.binary_crossentropy(y_true, y_pred)
    weighted_bce = weight * y_true * bce + (1 - y_true) * bce
    return K.mean(weighted_bce)

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        loss = weight * cross_entropy
        return K.mean(loss)
    return focal_loss_fixed

def boundary_loss(y_true, y_pred):
    def laplacian_kernel():
        return tf.constant([[0., 1., 0.],
                            [1., -4., 1.],
                            [0., 1., 0.]], shape=(3, 3, 1, 1), dtype=tf.float32)

    kernel = laplacian_kernel()
    y_true_edge = tf.nn.conv2d(y_true, kernel, strides=1, padding='SAME')
    y_pred_edge = tf.nn.conv2d(y_pred, kernel, strides=1, padding='SAME')
    diff = tf.abs(y_true_edge - y_pred_edge)
    return K.mean(diff)

def focal_dice_boundary_loss(y_true, y_pred, alpha=0.25, gamma=2.0, bce_weight=0.3, boundary_weight=0.1):
    fl = focal_loss(gamma=gamma, alpha=alpha)(y_true, y_pred)
    dl = dice_loss(y_true, y_pred)
    bl = boundary_loss(y_true, y_pred)
    return bce_weight * fl + (1 - bce_weight - boundary_weight) * dl + boundary_weight * bl

In [3]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim, hausdorff_distance
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import csv


def binarize_mask(mask):
    """Convert mask to binary (0 and 1)."""
    if mask.max() > 1:
        mask = mask // 255
    return mask.astype(np.uint8)

def compute_metrics(gt, pred):
    """Compute Dice, IoU, SSIM, Acc, Prec, Recall, F1, Hausdorff."""
    dice = (2. * np.sum(gt * pred)) / (np.sum(gt) + np.sum(pred) + 1e-6)
    iou = np.sum(gt & pred) / (np.sum(gt | pred) + 1e-6)
    ssim_val = ssim(gt, pred)

    flat_gt = gt.flatten()
    flat_pred = pred.flatten()
    acc = accuracy_score(flat_gt, flat_pred)
    prec = precision_score(flat_gt, flat_pred, zero_division=0)
    rec = recall_score(flat_gt, flat_pred, zero_division=0)
    f1 = f1_score(flat_gt, flat_pred, zero_division=0)

    # Hausdorff (skip if empty mask)
    if np.any(gt) and np.any(pred):
        haus = hausdorff_distance(gt, pred)
    else:
        haus = np.nan

    return dice, iou, ssim_val, acc, prec, rec, f1, haus

def save_to_csv(csv_path, model_name, image_name, metrics):
    """Append a line of metrics to CSV."""
    header = ['model_name', 'image_name', 'dice', 'iou', 'ssim', 'accuracy', 'precision', 'recall', 'f1_score', 'hausdorff']
    row = [model_name, image_name] + [round(m, 6) if not np.isnan(m) else '' for m in metrics]

    file_exists = os.path.isfile(csv_path)
    with open(csv_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(row)

In [4]:
import os
import cv2
from tqdm import tqdm
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s: %(message)s',
    datefmt='%H:%M:%S'
)


In [ ]:
import os
import cv2
from tqdm.notebook import tqdm

def evaluate_directory_pair(predicted_mask_dir, ground_truth_dir, model_name, output_csv):
    print(f"\n📂 Evaluating: {predicted_mask_dir}")
    print("📁 Loading predicted and ground truth files...")

    # Lookup for ground truth masks by filename
    gtruth_lookup = {}
    for f in os.listdir(ground_truth_dir):
        if f.lower().endswith((".png", ".jpg", ".jpeg")):
            base = os.path.splitext(f)[0]  # no extension
            gtruth_lookup[base] = os.path.join(ground_truth_dir, f)


    # Only include valid image files
    msk_files = [
        f for f in os.listdir(predicted_mask_dir)
        if f.endswith((".png", ".jpg", ".jpeg"))
    ]

    total, matched, failed = 0, 0, 0

    for msk in tqdm(msk_files, desc=f"Evaluating {model_name}", unit="img"):
        total += 1
        predicted_mask_path = os.path.join(predicted_mask_dir, msk)
        base_name = os.path.splitext(msk)[0]
        ground_truth_path = gtruth_lookup.get(base_name)


        if ground_truth_path is None:
            print(f"❌ No matching ground truth found for: {msk}")
            failed += 1
            continue

        pred = cv2.imread(predicted_mask_path, cv2.IMREAD_GRAYSCALE)
        gt = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)

        if pred is None or gt is None:
            print(f"❌ Failed to load images for: {msk}")
            failed += 1
            continue

        if pred.shape != gt.shape:
            print(f"⚠️ Shape mismatch: {msk} - pred: {pred.shape}, gt: {gt.shape}")
            failed += 1
            continue

        pred_bin = binarize_mask(pred)
        gt_bin = binarize_mask(gt)

        metrics = compute_metrics(gt_bin, pred_bin)
        save_to_csv(output_csv, model_name, msk, metrics)

        matched += 1

    print("✅ Evaluation complete.")
    print(f"📊 Total: {total}, ✅ Matched: {matched}, ❌ Failed/skipped: {failed}")
    print(f"📄 Results saved to: {output_csv}")


def main():
    base_path = "/content/gdrive/MyDrive"
    csv_dir = os.path.join(base_path, "Experimental_Results/Comparision_with_other_models/csv_results")
    os.makedirs(csv_dir, exist_ok=True)

    datasets = [
        ("SSL", "Experimental_Results/Comparision_with_other_models/SSL_outputs"),
        ("CellPose_SAM", "Experimental_Results/Comparision_with_other_models/CellPose_SAM_outputs"),
        ("StarDist", "Experimental_Results/Comparision_with_other_models/StarDist_outputs"),
        ("CellPose3", "Experimental_Results/Comparision_with_other_models/CellPose3_outputs"),
    ]

    # Assuming all share the same GT directory (update if needed)
    gt_dir = "/content/gdrive/MyDrive/Experimental_Results/Comparision_with_other_models/10_Lab_msk"

    for model_name, pred_rel in datasets:
        pred_dir = os.path.join(base_path, pred_rel)
        output_csv = os.path.join(csv_dir, f"{model_name}_results.csv")

        evaluate_directory_pair(pred_dir, gt_dir, model_name, output_csv)



In [ ]:
if __name__ == "__main__":
    main()